In [1]:
import pandas as pd

from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
okt = Okt()

In [3]:
# 데이터 가져오기
hotel_info_df = pd.read_csv('../../dataset/final/hotel_info.csv', encoding='utf-8')
hotel_review_df = pd.read_csv('../../dataset/final/hotel_review.csv', encoding='utf-8')

In [4]:
hotel_info_df.head(3)

,region,hotel_id,hote_name,hotel_link,hotel_img_link
0,서울,1,콘래드 서울,https://www.booking.com/hotel/kr/conrad-seoul....,https://t-cf.bstatic.com/xdata/images/hotel/sq...
1,서울,2,시그니엘,https://www.booking.com/hotel/kr/signiel-seoul...,https://t-cf.bstatic.com/xdata/images/hotel/sq...
2,서울,3,그랜드 하얏트 서울,https://www.booking.com/hotel/kr/grand-hyatt-s...,https://t-cf.bstatic.com/xdata/images/hotel/sq...


In [5]:
hotel_review_df.head(3)

,review_id,date,review,hotel_index,initial_label
0,1,2021-06,조식이 세미 뷔페식이었어요. \n여자친구는 건강식 전 펜케익을 먹었었는데 맛은 매우...,1,1
1,2,2022-01,우선 예약 상 문제가 있었던 부분에 대해 콘래드 측에서 즉각적인 대응을 해주셔서\n...,1,1
2,3,2022-02,쇼핑몰과 연결되어 있어 이동이 편했습니다.,1,1


In [6]:
hotel_review_df.rename(columns = {'hotel_index' : 'hotel_id'}, inplace=True)

In [2]:
def cleansing(document):
    # okt.tagset => 품사확인

    # document normalization
    norm_doc = okt.normalize(document)
    token_list = okt.pos(norm_doc, join=True)
    targets = ["Adjective", "Adverb", "Noun", "Verb"]

    # targets에 해당되는 token만 추출해 저장
    doc_tokenized = []
    for token in token_list:
        for target in targets:
            if target in token:
                dash_index = token.index("/")
                doc_tokenized.append(token[0:dash_index])

    # stopwords 제거한 token 저장
    stop_words = "이	있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 소리 놓"
    stop_words = set(stop_words.split(" "))
    cleansed_doc = [word for word in doc_tokenized if not word in stop_words]

    return cleansed_doc

In [9]:
# review_list를 가져와서 명사만 추출한 후 tag(index)와 함께 tagged_corpus_list에 저장
tagged_corpus_list = []
for df in hotel_review_df.itertuples():
    index = df.Index
    review_id = df.review_id
    hotel_id = df.hotel_id
    review = df.review

    cleansed_doc = cleansing(review)
    
    try:
        tagged_line = TaggedDocument(tags=[f'{hotel_id}@{review_id}'], words=cleansed_doc)
        tagged_corpus_list.append(tagged_line)
    except Exception as e:
        print('Error in tagged_corpus')
        continue
    print(f'---{round((index / len(hotel_review_df)) * 100 ,3)}%---')

---0.0%---
---0.001%---
---0.002%---
---0.004%---
---0.005%---
---0.006%---
---0.007%---
---0.009%---
---0.01%---
---0.011%---
---0.012%---
---0.014%---
---0.015%---
---0.016%---
---0.017%---
---0.019%---
---0.02%---
---0.021%---
---0.022%---
---0.024%---
---0.025%---
---0.026%---
---0.027%---
---0.028%---
---0.03%---
---0.031%---
---0.032%---
---0.033%---
---0.035%---
---0.036%---
---0.037%---
---0.038%---
---0.04%---
---0.041%---
---0.042%---
---0.043%---
---0.045%---
---0.046%---
---0.047%---
---0.048%---
---0.049%---
---0.051%---
---0.052%---
---0.053%---
---0.054%---
---0.056%---
---0.057%---
---0.058%---
---0.059%---
---0.061%---
---0.062%---
---0.063%---
---0.064%---
---0.066%---
---0.067%---
---0.068%---
---0.069%---
---0.071%---
---0.072%---
---0.073%---
---0.074%---
---0.075%---
---0.077%---
---0.078%---
---0.079%---
---0.08%---
---0.082%---
---0.083%---
---0.084%---
---0.085%---
---0.087%---
---0.088%---
---0.089%---
---0.09%---
---0.092%---
---0.093%---
---0.094%---
---0.09

In [12]:
tagged_corpus_list[:3]

[TaggedDocument(words=['조식', '세미', '뷔페', '이었어요', '여자친구', '건강', '펜', '케익', '먹었었는데', '맛', '매우', '만족스러웠어요', '고급스러운', '호텔', '조식', '원하신다면', '추천', '합니다'], tags=['1@1']),
 TaggedDocument(words=['우선', '예약', '상', '있었던', '부분', '대해', '콘래드', '측', '즉각', '대응', '해주셔서', '만족스러운', '서비스', '받은', '대해', '감사', '드립니다', '몰이', '지하', '바로', '연결', '되고', '이를', '통해', '현대', '지하', '이용', '할', '있어', '편안한', '쇼핑', '가능하며', '한강', '도보', '갈', '있을', '의', '거리', '입니다', '한강', '망', '가능한', '방', '이를', '보고', '있는', '편안한', '휴식', '되며', '실내', '수영장', '레인', '구분', '통해', '가족', '이용', '또는', '자유', '수영', '하기', '좋은', '환경', '이었습니다', '조식', '신선한', '재료', '사용', '했다는', '느낌', '받아', '만족', '이용', '했습니다'], tags=['1@2']),
 TaggedDocument(words=['쇼핑몰', '연결', '되어', '있어', '이동', '편했습니다'], tags=['1@3'])]

In [13]:
filtered_tagged_corpus_list = [x for x in tagged_corpus_list if (len(x[0]) >= 5) and (len(x[0]) <= 100)]

In [14]:
filtered_tagged_corpus_list[:3]

[TaggedDocument(words=['조식', '세미', '뷔페', '이었어요', '여자친구', '건강', '펜', '케익', '먹었었는데', '맛', '매우', '만족스러웠어요', '고급스러운', '호텔', '조식', '원하신다면', '추천', '합니다'], tags=['1@1']),
 TaggedDocument(words=['우선', '예약', '상', '있었던', '부분', '대해', '콘래드', '측', '즉각', '대응', '해주셔서', '만족스러운', '서비스', '받은', '대해', '감사', '드립니다', '몰이', '지하', '바로', '연결', '되고', '이를', '통해', '현대', '지하', '이용', '할', '있어', '편안한', '쇼핑', '가능하며', '한강', '도보', '갈', '있을', '의', '거리', '입니다', '한강', '망', '가능한', '방', '이를', '보고', '있는', '편안한', '휴식', '되며', '실내', '수영장', '레인', '구분', '통해', '가족', '이용', '또는', '자유', '수영', '하기', '좋은', '환경', '이었습니다', '조식', '신선한', '재료', '사용', '했다는', '느낌', '받아', '만족', '이용', '했습니다'], tags=['1@2']),
 TaggedDocument(words=['쇼핑몰', '연결', '되어', '있어', '이동', '편했습니다'], tags=['1@3'])]

In [15]:
# 모델 생성
d2v_model = Doc2Vec(vector_size=300, window=3, workers=8, min_count = len(tagged_corpus_list) // 1000)

In [16]:
# 단어 빌드
d2v_model.build_vocab(tagged_corpus_list)

In [17]:
# 학습
d2v_model.train(tagged_corpus_list, total_examples=d2v_model.corpus_count, epochs=30)

In [18]:
d2v_model.save('../model/d2v.model')

In [ ]:
d2v_model = Doc2Vec.load('./model/d2v.model')

In [18]:
print(len(filtered_tagged_corpus_list))
print(len(tagged_corpus_list))

79117
80851


In [3]:
# 테스트
user_input = '편의점 가까운'

cleansed_test_target = cleansing(user_input)
noun_user_input = cleansed_test_target
print(noun_user_input)

NameError: name 'okt' is not defined

In [20]:
# 유저 입력값의 벡터 생성
user_input_vector = d2v_model.infer_vector(noun_user_input)

In [21]:
# 유사도가 높은 리뷰 확인
most_similar_docs = d2v_model.docvecs.most_similar([user_input_vector], topn=len(d2v_model.docvecs))

In [22]:
most_similar_docs

[('62@5436', 0.7760463953018188),
 ('309@32106', 0.7509297132492065),
 ('441@59666', 0.7451207637786865),
 ('383@41180', 0.7436208724975586),
 ('539@77067', 0.7420017719268799),
 ('330@36131', 0.7396482229232788),
 ('430@58158', 0.7382370233535767),
 ('359@37921', 0.7358278036117554),
 ('46@3897', 0.7352207899093628),
 ('366@39222', 0.7348042726516724),
 ('263@21523', 0.7332783937454224),
 ('429@57260', 0.7328338623046875),
 ('87@6644', 0.728466808795929),
 ('73@6325', 0.7276451587677002),
 ('433@58370', 0.7268866896629333),
 ('476@62742', 0.7267146706581116),
 ('141@11780', 0.7230274081230164),
 ('425@56395', 0.7223118543624878),
 ('29@2686', 0.7209882140159607),
 ('44@3582', 0.7204475998878479),
 ('166@15025', 0.7190713882446289),
 ('422@54937', 0.714745283126831),
 ('322@34314', 0.712410569190979),
 ('550@77668', 0.7123048901557922),
 ('328@35653', 0.7098071575164795),
 ('330@36083', 0.7090546488761902),
 ('247@20556', 0.7084577083587646),
 ('378@40424', 0.7060520648956299),
 ('286@

In [23]:
# 몇 개의 호텔을 추천할지
number_top = 100

# 몇 개의 리뷰를 기준으로 할지
number_review = 4

# list의 길이를 기준으로 정렬하기 위해 Series 사용
similar_review_hotel = pd.Series([], dtype='object')

count_pass_hotel = 0
for review, similarity in most_similar_docs:
    hotel, index = review.split("@")

    if hotel not in similar_review_hotel:
        similar_review_hotel[hotel] = [index]
    else:
        similar_review_hotel[hotel].append(index)
    
    if len(similar_review_hotel[hotel]) == number_review:
        count_pass_hotel += 1
    if count_pass_hotel >= number_top:
        break

In [24]:
similar_review_hotel

62                                    [5436, 5463, 9966]
309    [32106, 32223, 32192, 31729, 43326, 32182, 323...
441    [59666, 59676, 59029, 59318, 59244, 59222, 697...
383           [41180, 41152, 52140, 52099, 41162, 52135]
539                                [77067, 80304, 77068]
                             ...                        
349                                       [36973, 36947]
338                                              [47970]
145                                              [23446]
233                                              [20098]
35                                                [2832]
Length: 287, dtype: object

In [25]:
recomended_hotel = similar_review_hotel.sort_values(key=lambda x: x.str.len(), ascending=False).head(number_top)

In [26]:
recomended_hotel

441    [59666, 59676, 59029, 59318, 59244, 59222, 697...
429    [57260, 57740, 57299, 57683, 68059, 57813, 577...
423    [55269, 55426, 55563, 55500, 55447, 65618, 554...
324    [35122, 46191, 34932, 46241, 46667, 34880, 463...
422    [54937, 54423, 55011, 54698, 54625, 64858, 548...
                             ...                        
405                         [53494, 42668, 42722, 53464]
253                         [21009, 20991, 29989, 29985]
546                         [77498, 80736, 77520, 80753]
215                         [18944, 18957, 28380, 28381]
421                         [64304, 54377, 54277, 64257]
Length: 100, dtype: object

In [27]:
# 추천된 호텔과 리뷰 확인
for hotel_id, review_id_list in list(recomended_hotel.items()):
    print(f'----- {hotel_info_df.loc[int(hotel_id),"hote_name"]} -----')
    for review_id in review_id_list[:number_review]:
        print(hotel_review_df.loc[int(review_id)]['review'])
    print('')

----- 씨크루즈호텔 속초 -----
바다뷰와 바다와 가까워서 산책하기 좋아요~~(바다쪽 방은 일출을 방에서 볼수 있어서 좋아요)
바다도 잘 보이고 깨끗하고 좋았습니다. 주차장 시설도 잘 되어있고 전체적으로 좋았어요.
위치가 좋음.  방이 깨끗하고 침대가 큼 해돋이 전망이 최고임
위치는 좋았으나 주변 상가환경이 다소 산만한 것이 아쉬움

----- 더화이트 호텔 -----
부킹닷컴 통해 할인 받았습니다 시설은 깨끗하고 좋습니다  천장이 높아서 넓게 느껴져 정말 좋았습니다
다락처럼 복층으로 되어있어 애들이 좋아할것 같아요 ~ (밑에층 이나 옆방들은 소음에 괴롭겠지만 ㅠ) 테라스에서 바라보는 뷰와 공기가 참 좋아요~ 위치도 괜찮고 가성비면에서는 나쁘지 않아요
가성비가 좋고 위치가 외진만큼 호텔내 부대시설이 잘되어있음. 씨유 포함 음식점 다수. 전체적인 위생상태가 깨끗하고 호텔인 만큼 호텔다운 구성을 갖춤 (어메니티 투숙객 서비스 등) 층고가 높아 객실이 작아도 답답하지 않고 침구류도 좋았음. 산속에 있어서 한여름에도 매우 시원하고 6월 말임에도 모기등 벌레가 별로 없었음. 양떼목장 등 관광지와도 가까움
층고가 높아서 확ㅡ트인 느낌과 깔끔한 분위기는 첫인상을 매우 좋게 만들어줍니다^^ 복층구조인데 ㅣㅣ살 조카와 함께했지만 굳이 올라갈 일은 없었네요. 근데 그런 거 있잖아요? 다락방있는 것만으로도 어른들에겐 옛운치 아이들에겐 나만의 공간ㅡ처음엔 거기가 자신의 아지트라고 하더니 허리를 굽혀 다녀야하는 2층이 불편했는지 더이상 오르지 않음^^ ㅡ고객의 감성을 신경쓴 느낌이라 아주 만족해했습니다. 아~그리고 조용해서 아주 좋았습니다~방배정 잘못 받으면 돈 내고 스트레스 쌓아서 가는 불운ㅜㅜ 아주 조용하고 시원하고 깔끔하고 함께 한 할머니와 조카군 모두 다음에 또 와보자 했을 정도입니다. 가격대비 최고라 감히 적어봅니다.

----- 롯데 리조트 속초 -----
해변과 가까이 있어서 산책하기가 매우 좋았음. 호텔 주위에 관광지가 가깝게 위치해 있어서 쉽게 관광지를 둘러 보

In [36]:
return_data = []
for hotel_id, review_id_list in recomended_hotel.items():
    hotel_dict = {}
    hotel_dict["hotel_id"] = hotel_id
    hotel_dict['review_id'] = []
 
    for review_id in review_id_list[:len(recomended_hotel[-1])]:
        #print(review_id)
        hotel_dict['review_id'].append(review_id)
    
    return_data.append(hotel_dict)

In [ ]:

# 추천된 호텔과 리뷰 확인
for hotel_id, review_id_list in return_data:
    print(f'----- {hotel_info_df.loc[int(hotel_id),"hote_name"]} -----')
    for review_id in review_id_list[:number_review]:
        print(hotel_review_df.loc[int(review_id)]['review'])
    print('')

In [63]:
return_data

[{'hotel_id': '441', 'review_id': ['59666', '59676', '59029', '59318']},
 {'hotel_id': '429', 'review_id': ['57260', '57740', '57299', '57683']},
 {'hotel_id': '423', 'review_id': ['55269', '55426', '55563', '55500']},
 {'hotel_id': '324', 'review_id': ['35122', '46191', '34932', '46241']},
 {'hotel_id': '422', 'review_id': ['54937', '54423', '55011', '54698']},
 {'hotel_id': '309', 'review_id': ['32106', '32223', '32192', '31729']},
 {'hotel_id': '518', 'review_id': ['75062', '78487', '74965', '78656']},
 {'hotel_id': '366', 'review_id': ['39222', '39419', '50404', '39377']},
 {'hotel_id': '455', 'review_id': ['60969', '71952', '71670', '71627']},
 {'hotel_id': '322', 'review_id': ['34314', '34421', '34466', '34420']},
 {'hotel_id': '442', 'review_id': ['59926', '59792', '59958', '70590']},
 {'hotel_id': '451', 'review_id': ['71163', '60610', '71518', '71151']},
 {'hotel_id': '353', 'review_id': ['48954', '37238', '37266', '37348']},
 {'hotel_id': '385', 'review_id': ['41529', '41510'

In [61]:


for i in range(len(return_data)):
    hotel_id = return_data[i]['hotel_id']
    review_id_list = return_data[i]['review_id']
    
    print(f'----- {hotel_info_df.loc[int(hotel_id),"hote_name"]} -----')

    for review_id in review_id_list[:number_review]:
        print(hotel_review_df.loc[int(review_id)]['review'])
    print('')


----- 씨크루즈호텔 속초 -----
바다뷰와 바다와 가까워서 산책하기 좋아요~~(바다쪽 방은 일출을 방에서 볼수 있어서 좋아요)
바다도 잘 보이고 깨끗하고 좋았습니다. 주차장 시설도 잘 되어있고 전체적으로 좋았어요.
위치가 좋음.  방이 깨끗하고 침대가 큼 해돋이 전망이 최고임
위치는 좋았으나 주변 상가환경이 다소 산만한 것이 아쉬움

----- 더화이트 호텔 -----
부킹닷컴 통해 할인 받았습니다 시설은 깨끗하고 좋습니다  천장이 높아서 넓게 느껴져 정말 좋았습니다
다락처럼 복층으로 되어있어 애들이 좋아할것 같아요 ~ (밑에층 이나 옆방들은 소음에 괴롭겠지만 ㅠ) 테라스에서 바라보는 뷰와 공기가 참 좋아요~ 위치도 괜찮고 가성비면에서는 나쁘지 않아요
가성비가 좋고 위치가 외진만큼 호텔내 부대시설이 잘되어있음. 씨유 포함 음식점 다수. 전체적인 위생상태가 깨끗하고 호텔인 만큼 호텔다운 구성을 갖춤 (어메니티 투숙객 서비스 등) 층고가 높아 객실이 작아도 답답하지 않고 침구류도 좋았음. 산속에 있어서 한여름에도 매우 시원하고 6월 말임에도 모기등 벌레가 별로 없었음. 양떼목장 등 관광지와도 가까움
층고가 높아서 확ㅡ트인 느낌과 깔끔한 분위기는 첫인상을 매우 좋게 만들어줍니다^^ 복층구조인데 ㅣㅣ살 조카와 함께했지만 굳이 올라갈 일은 없었네요. 근데 그런 거 있잖아요? 다락방있는 것만으로도 어른들에겐 옛운치 아이들에겐 나만의 공간ㅡ처음엔 거기가 자신의 아지트라고 하더니 허리를 굽혀 다녀야하는 2층이 불편했는지 더이상 오르지 않음^^ ㅡ고객의 감성을 신경쓴 느낌이라 아주 만족해했습니다. 아~그리고 조용해서 아주 좋았습니다~방배정 잘못 받으면 돈 내고 스트레스 쌓아서 가는 불운ㅜㅜ 아주 조용하고 시원하고 깔끔하고 함께 한 할머니와 조카군 모두 다음에 또 와보자 했을 정도입니다. 가격대비 최고라 감히 적어봅니다.

----- 롯데 리조트 속초 -----
해변과 가까이 있어서 산책하기가 매우 좋았음. 호텔 주위에 관광지가 가깝게 위치해 있어서 쉽게 관광지를 둘러 보